In [2]:
# Imports
# Basics
from __future__ import print_function, division
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
# gensim
from gensim import corpora, models, similarities, matutils
# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split


from sklearn.decomposition import NMF

# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

C:\Users\varru\Anaconda3\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Vector Space Models



Today, we'll map text documents from a highly dimensional word (or token) space into a much reduced **semantic space** which allows us to make valuable **conceptual comparisons** between arbitrary blocks of text in this new vector space.

Thus the **input is a large corpus of text documents** and the **output is a reduced semantic space for those input documents and words**.  
On a [**document-term matrix**](https://en.wikipedia.org/wiki/Document-term_matrix) with [**TFIDF Weightings**](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) to map all the terms in the corpus into a reduced **term space** and all the documents into a reduced **document space**.  
    - The 2 spaces are related by a simple transformation, so we can perform arbitrary **term-term**, **doc-doc**, and **doc-term comparisons** via [**cosine similarity**](https://en.wikipedia.org/wiki/Cosine_similarity).
    - These 2 spaces make up the **"dual space"**
        - Every document is the weighted sum of all of its terms
        - Every term is the weighted sum of all the documents it occurs in (very useful!)

[**Word2Vec**](https://en.wikipedia.org/wiki/Word2vec) - uses a neural network to yield **term space**
    - Has additional nice properties of term vectors, such as conceptual additivity (see below)

## Goals &zwnj;

- Use Word2vec to create a vector space for words in a training set
- Use the Word2vec space to do simple comparisons between different combinations of words
- Discuss various other considerations, tasks, and extensions for VSMs like LSI, NMF, and Word2vec

## Agenda
- Vector Space Models
  - What?
  - Why?
  - How?
- TFIDF

- Word2Vec

# What are Vector Space Models?
- Map raw text into vector space
- Allow semantic (conceptual) comparison between text chunks
- **Input**: Corpus of raw text documents
- **Output**: Vectors for text documents (and usually terms)

## Why do we need Vector Space Models?
- Early NLP (1950s-1980s): focused on **linguistics** and **handwritten rules**
  - Extremely complex, impossible to maintain/scale
- 1980s: ML introduced for NLP
  - Linguistics $\rightarrow$ "***Corpus Linguistics***"
  - Learn from text via large ***corpora*** of labeled raw text documents
- **Idea**: Map text to mathematical entities $\rightarrow$ ***Word Vectors***
- No more complex rules systems! $\rightarrow$ *Unsupervised*

## How do VSMs work?
- Start with raw text
- Translate text to vectors (e.g. Counts, TFIDF)
- Optional (Probable): **Reduce the space**
  - Use Probabilistic Inference (LDA)
  - Use Dimensionality Reduction via **Matrix Factorization** (SVD, NMF)
  - Use a Neural Network (Word2Vec)
- Once we have "Semantic" (meaning) vectors (**word vectors**):
  - Do all sorts of ML with them!

## Examples of VSMs
- **LDA**:
  - Word Space $\rightarrow$ Topic Space via Bayesian Inference
- **TFIDF**:
  - Word Space not reduced, but augmented
- **LSA/LSI** and **NMF**:
  - Word Space $\rightarrow$ Semantic Space via SVD or NMF 
- **Word2Vec**:
  - Word Space $\rightarrow$ Semantic Space via Neural Network

## Term Frequency Inverse Document Frequency (TFIDF)

## TFIDF
- Creates vectors for documents based on unique term counts (frequencies)
- Weights the frequency counts by TFIDF weighting
- Does **not** reduce dimensionality
- Frequent preprocessing step for matrix factorization methods

### Term-Document Matrix (TDM)
- Start with corpus of documents (raw text)
- Create matrix:
  - Rows are our **term vocabulary** - unique terms over all documents
  - Columns are all documents
  - Entries are respective term frequency for each document
- "Terms" means tokens, whatever is extracted from tokenization in preprocessing

### TFIDF Weighting
- Term Frequency Inverse Document Frequency
- Common weighting scheme applied to term-document matrix
- Any function that is:
  - Directly proportional to term frequency **within document** (local weight)
  - Inversely proportional to term frequency **in all documents** (global weight)
- **Motivation**: Highly common terms are not useful to distinguish documents

### TFIDF: Relation to VSMs
- TFIDF vectors are a VSM on their own! 
- TFIDF weightings empirically improve VSMs
- Some VSMs (LSI, NMF) almost certainly require this step, some don't (LDA, Word2Vec)

### Preprocessing Considerations
- Tokenization: Determines our unique terms $\rightarrow$ size of matrix
- Stopwords
- Stemming
- Named Entity Recognition
- Punctuation
- Min/Max Frequency Threshold (how often should term occur to keep it?)
- Phrase Extraction
- Part-of-Speech Tagging
- Word-sense Disambiguation (bush vs George Bush)
- etc

### Common TFIDF Weighting Scheme
- **Entropy**:
  - **Term Frequency** for term $t$ in a specific document: $tf_{t}$ i.e. $\frac{count(term_t)}{len(doc)}$
  - **Document Frequency** for term $t$: fraction of documents that contain term $t$: $df_t$ i.e. $\frac{n_t}{N}$, where $n_t$ is the number of documents that term t occurs in and N is the total number of documents in the corpus.
  - TFIDF = term-frequency **inverse** document frequency, usually with smoothing and log scaling
  - TFIDF Weight: 
$$
tfidf = (1 + \log{tf_t}) \log(\frac{1}{df_t}) 
$$

$$
\qquad \qquad = (1 + \log{tf_t}) \log(\frac{N}{n_t})
$$

The intuition behind tf-idf weighting is that frequent use of a word is only important relative to how common that word is across all documents. For example, if I have a document with the word "the" occuring a bunch of times, we shouldn't conclude much from that because "the" occurs in pretty much all the documents (**high tf but also high df**). But if I have a document with the word "correlation" occuring 50% of the time when it only occurs in 3% of documents (**high tf and low df**), I can now treat that as much more important information.

Tf-idf has an information theoretic/probabilistic interpretation as calculating the cross-entropy between the global distribution of a term across the corpus and the local distribution of the term in a document. [See here for more](http://searchivarius.org/blog/tf-idf-simply-cross-entropy).

**TFIDF in `sklearn`**: 
Let's implement TFIDF with the [20 Newsgroups Dataset](http://scikit-learn.org/stable/datasets/twenty_newsgroups.html)
- Here's TFIDF in `sklearn` with `TfidfVectorizer`:

In [3]:
# Load in the 20 Newsgroups data
ng = datasets.fetch_20newsgroups()
# Get the raw text docs
ng_text = ng.data

# Vectorize the text using TFIDF
tfidf = TfidfVectorizer(stop_words="english", 
                        token_pattern="\\b[a-zA-Z][a-zA-Z]+\\b", #words with >= 2 alpha chars 
                        min_df=10)
tfidf_vecs = tfidf.fit_transform(ng_text)
pd.DataFrame(tfidf_vecs.todense(), 
             columns=tfidf.get_feature_names()
            ).head()

,aa,aaa,aardvark,aaron,aau,ab,abandon,abandoned,abbreviation,abc,...,zoom,zq,zr,zs,zubov,zuma,zurich,zx,zyeh,zz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Applications of TFIDF
- We have a vector space!
  - Document vectors are the rows
  - Term vectors are the columns
- We can try to do ML
  - e.g.: Naive Bayes for Text Classification
- **BUT**...
  - Many unique terms $\rightarrow$ many unique features!
  - Curse of Dimensionality :(
  - Dimensionality Reduction seems prudent :)

### Naive Bayes Text Classification
- Naive Bayes empirically avoids the curse somewhat on TFIDF vectors
  - Performs decent enough on high-dimensional text classification tasks
- How does it work?
  - The observations are the weighted TFIDF vectors
    - This is a (weighted) bag of words for document $j$ and terms $\{w_i\}$
$$
P(Class C | \{w_i\}) = \frac{\text{Likelihood} \times \text{Prior}}{\text{Evidence}} = \frac{P(\{w_i\} | C) \times P(C)}{P(\{w_i\})}
$$
- Naive Assumption: 
$$
P(\{w_i\} | C) = \prod\limits_i P(w_i|C)
$$
- This is just a multinomial distribution where **given a class C each word has probability $p_i$ of appearing**!
- Thus:
  - **Likelihood is multinomial**:
    - $P(w_i)|C)$: Number of times word $w_i$ appears in class C documents divided by total number of words in Class C documents
  - **Prior**: Proportion of documents of class C
  - We can use Multinomial Naive Bayes!

Let's try simple Naive Bayes classification on TFIDF vectors from above in `sklearn`:

In [4]:
# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(tfidf_vecs, 
                                                    ng.target, 
                                                    test_size=0.33)

# Train 
nb = MultinomialNB()
nb.fit(X_train, y_train)

# Test 
nb.score(X_test, y_test)

0.86582753079807173

## Word2Vec

### What is word2vec?
- VSM for text analysis
- **Input**: Corpus of text documents
- **Output**: Reduced vector space for all terms(individual words) in documents, captures **Semantics** and **Syntax**.
- **Training**: Neural Network based on sliding windows of text

### Why word2vec?
- Other VSMs (LSI, NMF, etc) **don't capture word order** (Bag of Words)!
- It would be nice if we could, at least a little!
- word2vec does somewhat
- word2vec can capture analogy relationships:
  - e.g.: King is to man as Queen is to woman

### How does word2vec work?
- We use a **shallow** (1 hidden layer) neural network 
- Train on "**context windows**", small sequences of words in text (5-10 maybe)
- The input is either:
  - A word in the context window ("Skip-Grams") - **predicting the context given a word**
  - All words but 1 in a context window ("CBOW") - **predicting a word given the context**
- The output is either:
  - All the other words in a context window ("Skip-Grams")
  - The missing word from the context window ("CBOW")
  
So if we take a sentence: "Varun hopped into the TeslaCar" we would be taking one of these classification approaches:

* **Skip-Grams**: using into, predict Varun, hopped, the, and TeslaCar
* **CBOW**: using Varun, hopped, the, and Tesla, predict into.

### Preprocessing Considerations
- Everything from our previous VSMs!
- aka Tokenization, stemming, stopwords, Entity extraction, etc

### Training word2Vec
- Use a neural network on context windows
- 2 main approaches for inputs and labels:
  - **Skip-Grams**
  - **Continuous Bag of Words (CBOW)**
  - Vectors usually similar, subtle differences, also differences in computational time

#### Context Windows
- **Observations** for word2vec: All **context windows** in a corpus
  - Size of a context window is a chosen parameter
- e.g.:
  - Document: "The quick brown fox jumped over the lazy dog."
  - Window size: 5
  - Window 1: "**The quick brown fox jumped** over the lazy dog."
  - Window 2: "The **quick brown fox jumped over** the lazy dog."
  - Window 3: "The quick **brown fox jumped over the** lazy dog."
  - Window 4: "The quick brown **fox jumped over the lazy** dog."
  - Window 5: "The quick brown fox **jumped over the lazy dog**."

#### "One-hot" Encoding Word Vectors
- We need to be able to represent a **sequence of words** as a vector
- To do this, we need to assign each word an index from 0 to V
  - V is the size of the vocabulary aka # distinct words in the corpus
- A **word vector** is:
  - 1 for the index of that word
  - 0 for all other entries  
<img src='1-hot.png'/>

#### One-hot Encoding Context Windows
- We need vectors for context windows
- A sequence of words will have a vector that's just the concatenation of its word vectors
  - Thus, for window size $d$ the vector is of length $V \times d$
  - Only $d$ entries (one for each word) will be nonzero (1s)
  
<img src='catdog.png'/>

#### Skip-Grams
- Build a neural network with 1 hidden layer
- **Inputs**: 
  - The middle word of the context window (one-hot encoded)
  - Dimensionality: $V$
- **Outputs**: 
  - The other words of the context window (one-hot encoded)
  - Dimensionality: $V \times (d-1)$
- Turn the crank!  
<img src='skip_gram.png'/>

### Continuous Bag of Words (CBOW)
- Build a neural network with 1 hidden layer
- Just reverse of Skip-Grams!
- **Inputs**: 
  - The other words of the context window (one-hot encoded)
  - Dimensionality: $V \times (d-1)$
- **Outputs**: 
  - The middle word of the context window (one-hot encoded)
  - Dimensionality: $V$
- Turn the crank!  
<img src='cbow.png'/>

#### Dimensionality Reduction
- Number of nodes in hidden layer, $N$, is a parameter
- It is the (reduced) dimensionality of our resulting word vector space!
- Fit the neural net $\rightarrow$ find weights matrix $W$
  - In the new space, $x_N = W^Tx$
  - Checking dimensions:
    - $x$: $V \times 1$
    - $W^T$: $N \times V$
    - $x_N$: $N \times 1$

#### So What is Happening?
- We're learning the words likely to appear near each word
- This context information ultimately leads to vectors for related words falling near one another!

### Nice Properties of Word2Vec Vectors
- word2vec (somewhat magically!) captures nice geometric relations between words
<img src='vector_queen2.png' align='right'/>
- e.g.: Analogies
  - King is to Queen as man is to woman
  - The vector between King and Queen is the same as that between man and woman!
- Works for all sorts of things: capitals, cities, etc

### Word2Vec for ML
- Again we get **word vectors**!
- So we can use them for ML!



#### Using Existing Word Vectors
- word2vec takes **A LOT** of data to train it well
- What if you don't have **A LOT** of data?
  - Steal someone else's vectors!
  - Google has trained a [giant set](https://code.google.com/p/word2vec/)
  - So has [Stanford NLP](http://nlp.stanford.edu/projects/glove/) (slightly different, same idea)
  - Many others
- As long as the domain is similar, should be better than yours
  - Need words to have the same meaning as in your dataset

#### word2vec in `gensim`
- Very simple example for usage
- We'll train our own (don't!  steal vectors!)
- It needs a list of lists representing the sentences in a corpus
- Here's how that goes:

In [25]:
# Make sure gensim and Word2Vec are installed and functional
# Create some dummy data
sentences = documents = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",
              "Relation of user perceived response time to error measurement",
              "The generation of random binary unordered trees",
              "The intersection graph of paths in trees",
              "Graph minors IV Widths of trees and well quasi ordering",
              "Graph minors A survey"]

# The type of input that Word2Vec is looking for.. 
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in documents]

print(texts)

[['human', 'machine', 'interface', 'lab', 'abc', 'computer', 'applications'], ['survey', 'user', 'opinion', 'computer', 'system', 'response', 'time'], ['eps', 'user', 'interface', 'management', 'system'], ['system', 'human', 'system', 'engineering', 'testing', 'eps'], ['relation', 'user', 'perceived', 'response', 'time', 'error', 'measurement'], ['generation', 'random', 'binary', 'unordered', 'trees'], ['intersection', 'graph', 'paths', 'trees'], ['graph', 'minors', 'iv', 'widths', 'trees', 'well', 'quasi', 'ordering'], ['graph', 'minors', 'survey']]


In [26]:
# Train word2vec
w2v = models.Word2Vec(texts, size=100, window=5, min_count=1, workers=4,sg=1)
# Check out the resulting vector for "computer"
w2v['computer']

2018-06-01 22:26:07,672 : INFO : collecting all words and their counts
2018-06-01 22:26:07,672 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-06-01 22:26:07,672 : INFO : collected 35 word types from a corpus of 52 raw words and 9 sentences
2018-06-01 22:26:07,672 : INFO : Loading a fresh vocabulary
2018-06-01 22:26:07,672 : INFO : min_count=1 retains 35 unique words (100% of original 35, drops 0)
2018-06-01 22:26:07,672 : INFO : min_count=1 leaves 52 word corpus (100% of original 52, drops 0)
2018-06-01 22:26:07,672 : INFO : deleting the raw counts dictionary of 35 items
2018-06-01 22:26:07,672 : INFO : sample=0.001 downsamples 35 most-common words
2018-06-01 22:26:07,688 : INFO : downsampling leaves estimated 11 word corpus (21.7% of prior 52)
2018-06-01 22:26:07,688 : INFO : estimated required memory for 35 words and 100 dimensions: 45500 bytes
2018-06-01 22:26:07,688 : INFO : resetting layer weights
2018-06-01 22:26:07,688 : INFO : training model wit

array([  1.65059741e-04,  -3.45909828e-03,  -7.98419351e-05,
         1.47965609e-03,  -2.72638397e-03,   5.03118266e-04,
        -2.71530892e-03,  -1.54730002e-03,   1.33191678e-03,
         3.11394455e-03,   2.37710075e-03,  -1.93741685e-03,
        -2.04316876e-03,   3.37618752e-03,  -5.85331918e-07,
        -3.17614898e-03,   2.09084200e-03,  -9.49236564e-04,
        -3.75025650e-03,  -3.27538364e-05,  -3.04771168e-03,
         4.74398863e-03,  -3.69384652e-03,  -1.26163941e-03,
        -4.41896170e-03,  -3.53475707e-03,  -3.99893522e-03,
         3.46871163e-03,  -1.51816325e-03,   3.19902855e-03,
        -2.94388062e-03,  -4.29868745e-03,  -3.87685490e-03,
         2.84234085e-03,   4.10572626e-03,  -4.53026406e-03,
         4.05600807e-03,   3.75391566e-03,   6.45549502e-04,
        -2.98501947e-03,   4.64301789e-03,   1.36235240e-03,
         3.65666370e-03,   2.85392255e-03,  -2.83774198e-03,
        -1.41713172e-04,   4.12686961e-03,  -2.78024725e-03,
        -1.20313047e-03,

**Now on a slightly Larger Corpus**:  
- Using Project Gutenberg Corpus (Books) from NLTK:

In [27]:
# An Illustration.. 

import os

# Create an iterator Class that can be used as a gensim corpus (defines how to read in the text data)
class MySentences(object):
     def __init__(self, dirname):
        self.dirname = dirname
 
     def __iter__(self):
         for fname in os.listdir(self.dirname):
                for line in open(
                    os.path.join(self.dirname, fname), 
                    encoding='utf-8', errors='ignore'):
                    yield line.split()

# Instantiate the corpus from a text file of documents
# You'll need to change the path!
sentences = MySentences('/Users/varru/nltk_data/corpora/gutenberg') # a memory-friendly iterator
# Create a Word2vec model
w2v = models.Word2Vec(sentences,min_count=3,workers=5)

2018-06-01 22:26:08,421 : INFO : collecting all words and their counts
2018-06-01 22:26:08,421 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-06-01 22:26:08,468 : INFO : PROGRESS: at sentence #10000, processed 93960 words, keeping 12185 word types
2018-06-01 22:26:08,515 : INFO : PROGRESS: at sentence #20000, processed 189593 words, keeping 19551 word types
2018-06-01 22:26:08,546 : INFO : PROGRESS: at sentence #30000, processed 278383 words, keeping 24115 word types
2018-06-01 22:26:08,577 : INFO : PROGRESS: at sentence #40000, processed 359433 words, keeping 27838 word types
2018-06-01 22:26:08,608 : INFO : PROGRESS: at sentence #50000, processed 443441 words, keeping 33634 word types
2018-06-01 22:26:08,655 : INFO : PROGRESS: at sentence #60000, processed 532123 words, keeping 36348 word types
2018-06-01 22:26:08,687 : INFO : PROGRESS: at sentence #70000, processed 618856 words, keeping 39209 word types
2018-06-01 22:26:08,718 : INFO : PROGRESS: at s

#### Calculating Similarities
- Have word vectors for all terms!  
- Gensim provides a number of methods to then do comparisons in this vector space 
- Here are a few:

In [28]:
# Words close to woman and king but not man
w2v.most_similar(positive=['woman', 'king'], negative=['man'], topn=10)

C:\Users\varru\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
2018-06-01 22:26:21,379 : INFO : precomputing L2-norms of word weight vectors


[('Moses', 0.760481059551239),
 ('Solomon', 0.7598444223403931),
 ('David', 0.7492145299911499),
 ('captain', 0.7483667135238647),
 ('son', 0.7464020848274231),
 ('Joshua', 0.723465621471405),
 ('elders', 0.7191511988639832),
 ('Jesus', 0.7033897638320923),
 ('daughter', 0.7005856037139893),
 ('messengers', 0.6951842308044434)]

**Not so hot!**  
- Unsurprising: Need much more data
- A lesson in why you should steal vectors :)
- Let's try some pairwise comparisons:

In [19]:
# Similarity between woman and man
print(w2v.similarity('woman','man'))

# Similarity between bags of words
print(w2v.n_similarity(['woman', 'girl'], ['man', 'boy']))

# Finding words that don't match others in a bag
print(w2v.doesnt_match("breakfast man dinner lunch".split()))

0.755077698653
0.796434537654
man


C:\Users\varru\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  
C:\Users\varru\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """
C:\Users\varru\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  


#### Word2Vec for Text Clustering
- We won't do it here, but the procedure is similar to LSI above. The key difference is that now we **only have vectors for each word** instead of vectors for entire documents. We could get back to the document level by averaging across all of the word vectors in each document that we extract from our gensim model.

 - Try whatever clustering algorithm you like on the reduced document vectors
- **OR** more likely you just take the vectors as given from Google or Stanford and do the same thing

#### Word2Vec for Text Classification
- We won't do it here, but the procedure is similar to LSI above. The key difference is that now we **only have vectors for each word** instead of vectors for entire documents. We could get back to the document level by averaging across all of the word vectors in each document that we extract from our gensim model.

 - Try whatever classifying algorithm you like on the reduced vectors
    - For example, KNN with Cosine Similarity
- **OR** more likely you just take the vectors as given from Google or Stanford and do the same thing

## Applications of Word Vectors (VSMs) 
- With word vectors, we can do so many cool things:
  - ML algorithms
  - Machine Translation
  - Many of those things mentioned on NLP day 1
  - Seed Deep Learning with them to do **even cooler stuff**
- Basically, we know the state of the world (the meaning of words)...
  - The possibilities are endless!